# Essential TitaniumCloud file analysis and reputation API-s
This notebook contains specific examples for the essential TitaniumCloud file reputation and analysis-related API-s.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API classes:
- **FileReputation** (*TCA-0101 - File Reputation (Malware Presence)*)
- **AVScanners** (*TCA-0103 - Historic Multi-AV Scan Records*)
- **FileAnalysis** (*TCA-0104 - File Analysis - Hash*)
- **RHA1FunctionalSimilarity** (*TCA-0301 - RHA Functional Similarity (Group by RHA1)*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API classes from the ticloud module.

In [13]:
from ReversingLabs.SDK.ticloud import FileReputation, AVScanners, FileAnalysis, RHA1FunctionalSimilarity

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

Also, we will store the SHA-1 hash of a file we want to investigate into a variable called `FILE_HASH`.

In [14]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")
USER_AGENT = "ReversingLabs SDK Cookbook v1.0.2"

FILE_HASH = "21841b32c6165b27dddbd4d6eb3a672defe54271"

### 3. Fetching the file reputation
Next, we will create our first object out of the FileReputation class: file_reputation

In [15]:
file_reputation = FileReputation(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

We will now use our `file_reputation` object to do a reputation lookup for the hash of our potentially malicious file.

In [ ]:
reputation = file_reputation.get_file_reputation(
    hash_input=FILE_HASH
)

reputation_text = reputation.text
print(reputation_text)

Now we have the file reputation response stored in the `reputation` variable and also printed out in the console in text form.
Among other information, from the file reputation response we can see the following:
- This file is malicious - `"status": "MALICIOUS"`
- Its threat name is Win32.Ransomware.Tox - `"threat_name": "Win32.Ransomware.Tox"`
- There is a certain number of AV scanners, out of a total number of available AV scanners, that declared this file as harmful - `"scanner_match": 35`

### 4. Fetching the AV scanner results
These results provide malware detection verdicts of multiple antivirus scanners from the TitaniumCloud.
To see which AV scanners declared the file as harmful, we can use the AVScanners class.
To do so, we will again create an object:

In [17]:
av_scanners = AVScanners(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

Now that we have our `av_scanners` object, we can fetch the AV scanner results from the API.

In [ ]:
scanner_results = av_scanners.get_scan_results(
    hash_input=FILE_HASH
)

results_text = scanner_results.text
print(results_text)

To see which scanners were available, navigate to `"rl" -> "sample -> "xref" -> "scanners"`  
To see the verdict returned by each scanner, navigate to `"rl" -> "sample -> "xref" -> "results"`  

Other useful information included in the AV scanners report:
- `"sample_type": "PE32 executable (GUI) Intel 80386 (stripped to external PDB), for MS Windows, UPX compressed"`
- `"sample_size": 636416`

### 5. Fetching the file analysis results
To view the static analysis details of our malicious file, we can use the FileAnalysis class:

In [ ]:
analysis = FileAnalysis(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

analysis_results = analysis.get_analysis_results(
    hash_input=FILE_HASH
)

analysis_text = analysis_results.text
print(analysis_text)

By reading the analysis results, we can get the following useful information:
- `"sample_size": 636416`
- `"tc_report"` - a static analysis report created by TitaniumCore
    - `"file_type": "PE"`
    - `"file_subtype": "Exe"`
    - `"story"` - a verbose and human-readable description of this file
    - `"dynamic_analysis"` - dynamic analysis results from various cloud sandboxes


### 6. Getting the hashes of functionally similar files
Additionally, we can send use the RHA1FunctionalSimilarity class to get a list of files that are functionally similar to the malicious file that we are analyzing.  
Each file is represented by its SHA-1 hash.

In [ ]:
similarity = RHA1FunctionalSimilarity(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

similarity_results = similarity.get_similar_hashes(
    hash_input=FILE_HASH
)

similarity_text = similarity_results.text
print(similarity_text)

This gives us a report with only the first page of results. If we want a **complete** list of similar files, we need to use the `get_similar_hashes_aggregated` method.  
The full list is returned as a Python `list` type, rather than a response object.
Example:

In [ ]:
all_pages = similarity.get_similar_hashes_aggregated(
    hash_input=FILE_HASH
)

print(all_pages)


### 7. Advanced
**Additional parameters** in that method give us the following options:
- `extended_results` - show the extended results
- `classification` - return only files of this classification
- `max_results` - the maximum number of files we want
- `results_per_page` - the number of files per each page (request) we want, until we reach the `max_results`

Example:

In [ ]:
all_pages = similarity.get_similar_hashes_aggregated(
    hash_input=FILE_HASH,
    extended_results=True,
    classification="MALICIOUS",
    max_results=300,
    results_per_page=50
)

print(all_pages)

This example will return a maximum of 300 files in a list, with 50 files per page (request). Also, it will return only malicious files with extended information.